# **PFIRS Enrichment Processing**

## **Install Required Python Libraries**

You are editing this notebook, so we assume you have already cloned the repository from  
[https://github.com/klinucsd/interagency-tracking-system](https://github.com/klinucsd/interagency-tracking-system) into the folder ~/work/interagency-tracking-system. Please verify that the `requirements.txt` file is available in that folder before proceeding.

In [1]:
# ! pip install -r requirements.txt

## **Download PFIRS data in zip format**

In [3]:
from utils.download_from_box import download_box_shared_link

pfirs_box_link = "https://calfire.box.com/s/jpe3gz0y2y7kub7cq7lmv5mga9yc03su"
download_box_shared_link(pfirs_box_link)

Accessing shared link...
Found 1 file(s)

[1/1] Downloading: PFIRS2024.gdb.zip
    ✓ Saved: PFIRS2024.gdb.zip (989,726 bytes)



['./PFIRS2024.gdb.zip']

## **Unzip PFIRS data in GDB format**

In [4]:
! unzip PFIRS2024.gdb.zip

Archive:  PFIRS2024.gdb.zip
  inflating: PFIRS2024.gdb/a00000001.freelist  
  inflating: PFIRS2024.gdb/a00000001.gdbindexes  
  inflating: PFIRS2024.gdb/a00000001.gdbtable  
  inflating: PFIRS2024.gdb/a00000001.gdbtablx  
  inflating: PFIRS2024.gdb/a00000001.TablesByName.atx  
  inflating: PFIRS2024.gdb/a00000002.gdbtable  
  inflating: PFIRS2024.gdb/a00000002.gdbtablx  
  inflating: PFIRS2024.gdb/a00000003.gdbindexes  
  inflating: PFIRS2024.gdb/a00000003.gdbtable  
  inflating: PFIRS2024.gdb/a00000003.gdbtablx  
  inflating: PFIRS2024.gdb/a00000004.CatItemsByPhysicalName.atx  
  inflating: PFIRS2024.gdb/a00000004.CatItemsByType.atx  
  inflating: PFIRS2024.gdb/a00000004.FDO_UUID.atx  
  inflating: PFIRS2024.gdb/a00000004.gdbindexes  
  inflating: PFIRS2024.gdb/a00000004.gdbtable  
  inflating: PFIRS2024.gdb/a00000004.gdbtablx  
  inflating: PFIRS2024.gdb/a00000004.horizon  
  inflating: PFIRS2024.gdb/a00000004.spx  
  inflating: PFIRS2024.gdb/a00000005.CatItemTypesByName.atx  
  infl

## **Check the PFIRS GDB to Find Its Layers**

In [5]:
! ogrinfo -so PFIRS2024.gdb

INFO: Open of `PFIRS2024.gdb'
      using driver `OpenFileGDB' successful.
Layer: PFIRS2024 (Point)


## **Download Lookup Table**

In [12]:
lookup_box_link = "https://calfire.box.com/s/opn58bkf1iirnj25pducpx6zdf6xucjs"
download_box_shared_link(lookup_box_link)

Accessing shared link...
Found 1 file(s)

[1/1] Downloading: PFIRS_2023_20240624_dataJasonBranz_gdbByEricRowell.gdb.zip
    ✓ Saved: PFIRS_2023_20240624_dataJasonBranz_gdbByEricRowell.gdb.zip (1,867,971 bytes)



['./PFIRS_2023_20240624_dataJasonBranz_gdbByEricRowell.gdb.zip']

## **Unzip Reference Data in GDB format**

In [5]:
! unzip ../a_Reference.gdb.zip 

Archive:  ../a_Reference.gdb.zip
replace a_Reference.gdb/a00000007.CatRelTypesByBackwardLabel.atx? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


## **Check the Reference GDB to Find Its Layers**

In [6]:
! ogrinfo -so a_Reference.gdb

INFO: Open of `a_Reference.gdb'
      using driver `OpenFileGDB' successful.
Layer: Crosswalk (None)
Group a_Reference:
  Layer: WFR_TF_Template (Multi Polygon)
  Layer: California (Multi Polygon)
  Layer: Counties (Multi Polygon)
  Layer: Broad_Vegetation_Types (Multi Polygon)
  Layer: CALFIRE_Ownership_Update (Multi Polygon)
  Layer: WUI (Multi Polygon)
  Layer: WFRTF_Regions (Multi Polygon)
  Layer: Own_Veg_Region_WUI (Multi Polygon)
Group b_Originals:
Group c_Enriched:
Group d_Appended:
Group e_Transformed:
Group f_Report_Data:


## **Enriching PFIRS Data**

Converts PFIRS actual treatments dataset into the Task Force standardized schema. Dataset is enriched with vegetation, ownership, county, WUI, Task Force Region, and year. 

In [1]:

import warnings
import logging
import time
from datetime import datetime
import psutil
import os
from enrich.enrich_PFIRS import enrich_PFIRS

# Get the current process ID
process = psutil.Process(os.getpid())

pfirs_input_gdb_path = "PFIRS2024.gdb"
pfirs_input_layer_name = "PFIRS2024"
treat_poly_gdf_path = ""
treat_poly_layer_name = ""
a_reference_gdb_path = "a_Reference.gdb"
start_year, end_year = 2010, 2025
output_gdb_path = f"/tmp/PFIRS_{start_year}_{end_year}.gdb"
output_layer_name = f"PFIRS_enriched_{datetime.today().strftime('%Y%m%d')}"

enrich_PFIRS(pfirs_input_gdb_path,
             pfirs_input_layer_name,
             None,
             a_reference_gdb_path,
             start_year,
             end_year,
             output_gdb_path,
             output_layer_name)

# Get memory usage in bytes, convert to MB
memory_usage = process.memory_info().rss / 1024 / 1024
print(f"Memory usage: {memory_usage:.2f} MB")
print(f"PFIRS enrichment GDB: {output_gdb_path}")
print(f"PFIRS enrichment Layer: {output_layer_name}")

2025-12-19 02:09:09,348 INFO  [enrich.enrich_PFIRS   ]  Load the PFIRS data into a GeoDataFrame
2025-12-19 02:09:09,577 INFO  [enrich.enrich_PFIRS   ]     time for loading PFIRS2024: 0.22857022285461426
2025-12-19 02:09:09,639 INFO  [enrich.enrich_PFIRS   ]  Performing Standardization
2025-12-19 02:09:09,641 INFO  [enrich.enrich_PFIRS   ]     step 1/8 remove some agencies
2025-12-19 02:09:09,650 INFO  [enrich.enrich_PFIRS   ]     step 2/8 rename fields
2025-12-19 02:09:09,653 INFO  [enrich.enrich_PFIRS   ]     step 3/8 adding common columns...
2025-12-19 02:09:09,697 INFO  [enrich.enrich_PFIRS   ]     step 4/8 import attributes
2025-12-19 02:09:09,703 INFO  [enrich.enrich_PFIRS   ]        standardized has 3682 records
2025-12-19 02:09:09,704 INFO  [enrich.enrich_PFIRS   ]     step 5/8 remove points that intersect burn polygons
2025-12-19 02:09:09,705 INFO  [enrich.enrich_PFIRS   ]     step 6/8 Remove Unnecessary Columns...
2025-12-19 02:09:09,710 INFO  [enrich.enrich_PFIRS   ]  Perform

Memory usage: 6261.32 MB
PFIRS enrichment GDB: /tmp/PFIRS_2010_2025.gdb
PFIRS enrichment Layer: PFIRS_enriched_20251219
